<center>
  <img src="https://digip.unibg.it/sites/dip3/files/logo-dip-it.svg">
  <h1><b>Crash course in Python </b> - AA 2024/2025</h1>
  <h2>Lezione 3 - Caso di studio ✏️</h2>
  <h3>A cura di <a href="https://github.com/lamferzon?tab=repositories">Lorenzo Leoni</a></h3>
</center>

# Importazione dei pacchetti 📄

In [ ]:
import re
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import het_breuschpagan
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Importazione dei dati 💾

In [ ]:
data = pd.read_csv("phone_data.csv")

# rinominazione delle colonne
data = data.rename(
    columns={col_i: col_i.lower().replace(" ", "_") for col_i in data.columns}
)

# alcune informazioni riguardanti il dataset
nrows, ncols = data.shape
print(f"Dimensioni del dataset \n- Numero di righe: {nrows}.\n- Numero di colonne: {ncols}.")
print("\nColonne del dataset e rispettivi tipi")
for col_i in data.columns:
    print(f"- {col_i.capitalize()} ({data[col_i].dtype}).")

# Elaborazione preliminare 🔧

## Rimozione delle colonne superflue o ad alta cardinalità

In [ ]:
print("Cardinalità delle colonne categoriche")
for col_i in data.columns:
    if data[col_i].dtype == object:
        print(f"- {col_i.capitalize()}: {len(data[col_i].unique())}.")

data = data.drop(
    columns=[
        "phone_model",
        "price",
        "currency",
        "launch",
        "display_type",
        "display_resolution",
        "os",
        "usb",
        "features_sensors",
        "colors",
        "video",
        "gpu",
        "quantile_10",
        "quantile_50",
        "quantile_90"
    ]
)

## Creazione della colonna `volume`

In [ ]:
def get_volume(row):
    match = re.findall(r"([\d.]+) x ([\d.]+) x ([\d.]+) mm", row.dimensions)
    if match:
        height, width, thickness = match[0]
        return round(float(height)*float(width)*float(thickness), 2)
    else:
        None
data["volume"] = data.apply(get_volume, axis=1)

# rimozione della colonna "dimensions" e dei dati mancanti
data = data.drop(
    columns=[
        "dimensions"
    ]
).dropna().reset_index(drop=True)

## Creazione della colonna `chipset_brand`

In [ ]:
def get_chipset_brand(row):
    return row.chipset.split()[0]
data["chipset_brand"] = data.apply(get_chipset_brand, axis=1)

## Creazione della colonna `chipset_production_process`

In [ ]:
def get_chipset_production_process(row):
    match = re.search(r"\((\d+)\s*nm\)", row.chipset)
    if match:
        return int(match.group(1))
    else:
        None
data["chipset_production_process"] = data.apply(get_chipset_production_process, axis=1)

# rimozione della colonna "chipset" e dei dati mancanti
data = data.drop(
    columns=[
        "chipset"
    ]
).dropna().reset_index(drop=True)

## Creazione della colonna `cpu_cores`

In [ ]:
core_map = {
    'Hexa': 6,
    'Octa': 8,
    'Quad': 4,
    'Nona': 9,
    'Deca': 10,
    'Dual': 2
}

def get_cpu_cores(row):
    
    # primo passa: ricerca dei prefissi come "Hexa", "Octa", "Quad", etc.
    for key, value in core_map.items():
        if key.lower() in row.cpu.lower():
            return int(value)
    
    # secondo passo: utilizzo di un'espressione regolare (regex) per cercare numeri seguiti da "-core"
    match = re.search(r'(\d+)-core', row.cpu, re.IGNORECASE)
    if match:
        return int(match.group(1))
    else:
        return None

data["cpu_cores"] = data.apply(get_cpu_cores, axis=1)

# rimozione della colonna "cpu" e dei dati mancanti
data = data.drop(
    columns=[
        "cpu"
    ]
).dropna().reset_index(drop=True)

## Conversione di tipo delle colonne `nfc`, `foldable` e `year`

In [ ]:
data["phone_brand"] = data.phone_brand.str.capitalize()
data["nfc"] = data.nfc.replace(1, "Yes").replace(0, "Not")
data["foldable"] = data.foldable.replace(1, "Yes").replace(0, "Not")
data["year"] = data.year.astype(str)

## Classificazione delle variabili

In [ ]:
target_var = "price_usd"
num_vars = [var_i for var_i in data.columns if ((data[var_i].dtype == float) | (data[var_i].dtype == int)) & (var_i != target_var)]
cat_vars = [var_i for var_i in data.columns if data[var_i].dtype == object]

## DataFrame dopo il processamento iniziale

In [ ]:
data.sort_index(axis=1).sort_values(by="price_usd", ascending=False).head(10)

# Analisi preliminare 🔬

In [ ]:
fig, axes = plt.subplots(
    ncols=2,
    nrows=3,
    figsize=(15, 18)
)
fig.subplots_adjust(
    wspace=.2,
    hspace=.3
)

# andamento temporale del prezzo medio, per segmento di mercato
sns.lineplot(
    data=data,
    x="year",
    y="price_usd",
    ls="-.",
    hue="price_range",
    palette="bright",
    ax=axes[0, 0]
)
axes[0, 0].axvline(
    "2020",
    color="red",
    ls="--",
    alpha=.75,
    label="Pandemia"
)
axes[0, 0].set_title(
    "$\\mathbf{Andamento \\ temporale \\ del \\ prezzo \\ medio \\ di \\ vendita}$\n per segmento di mercato",
    size=11
)
axes[0, 0].set(
    xlabel="Anno",
    ylabel="Prezzo di vendita[$]",
    facecolor="whitesmoke"
)
axes[0, 0].grid(
    ls="--",
    alpha=.5
)
axes[0, 0].legend(
    title="$\\mathbf{Segmento}$"
)

# andamento temporale del prezzo medio, per negozio online
sns.lineplot(
    data=data,
    x="year",
    y="price_usd",
    hue="store",
    style="store",
    palette="bright",
    err_style=None,
    markers=True,
    dashes=False,
    ax=axes[0, 1]
)
axes[0, 1].set_title(
    "$\\mathbf{Andamento \\ temporale \\ del \\ prezzo \\ medio \\ di \\ vendita}$\nper negozio online",
    size=11
)
axes[0, 1].set(
    xlabel="Anno",
    ylabel="Prezzo di vendita [$]",
    facecolor="whitesmoke"
)
axes[0, 1].grid(
    ls="--",
    alpha=.5
)
axes[0, 1].legend(
    title="$\\mathbf{Negozio \\ online}$"
)

# andamento temporale del prezzo medio, per produttore
sns.lineplot(
    data=data[
        (data.phone_brand == "Samsung") | 
        (data.phone_brand == "Google") |
        (data.phone_brand == "Apple") |
        (data.phone_brand == "Xiaomi")
    ],
    x="year",
    y="price_usd",
    hue="phone_brand",
    palette="bright",
    err_style="bars",
    ax=axes[1, 0]
)
axes[1, 0].set_title(
    "$\\mathbf{Andamento \\ temporale \\ del \\ prezzo \\ medio \\ di \\ vendita}$\nper produttore",
    size=11
)
axes[1, 0].set(
    xlabel="Anno",
    ylabel="Prezzo di vendita [$]",
    facecolor="whitesmoke"
)
axes[1, 0].grid(
    ls="--",
    alpha=.5
)
axes[1, 0].legend(
    title="$\\mathbf{Produttore}$"
)

# heatmap del prezzo medio di vendita, per produttore
sns.heatmap(
    data.pivot_table(
        index="phone_brand",
        columns="year",
        values="price_usd",
        aggfunc="mean"
    ),
    annot=True,
    fmt=f".0f",
    cbar=False,
    ax=axes[1, 1]
)
axes[1, 1].set_title(
    "$\\mathbf{Heatmap\\ del \\ prezzo \\ medio \\ di \\ vendita}$\nper produttore",
    size=11
)
axes[1, 1].set(
    xlabel="Anno",
    ylabel=None
)

# distribuzione del prezzo medio, per segmento di mercato
sns.boxenplot(
    data=data,
    x="price_usd",
    hue="price_range",
    palette="bright",
    ax=axes[2, 0]
)
axes[2, 0].set_title(
    "$\\mathbf{Distribuzione \\ del \\ prezzo \\ di \\ vendita}$\nper segmento di mercato",
    size=11
)
axes[2, 0].set(
    xlabel="Prezzo di vendita [$]",
    facecolor="whitesmoke"
)
axes[2, 0].grid(
    axis="x",
    ls="--",
    alpha=.5
)
axes[2, 0].legend(
    title="$\\mathbf{Segmento}$"
)

# andamento temporale del numero di modelli rilasciati, per segmento di mercato (solo Amazon UK)
sns.lineplot(
    data=data[data["store"] == "Amazon UK"].groupby(["year", "price_range"]).agg(count=("price_range", "count")).reset_index(),
    x="year",
    y="count",
    hue="price_range",
    palette="bright",
    alpha=.5,
    legend=False,
    ax=axes[2, 1]
)
sns.scatterplot(
    data=data[data["store"] == "Amazon UK"].groupby(["year", "price_range"]).agg(count=("price_range", "count")).reset_index(),
    x="year",
    y="count",
    hue="price_range",
    palette="bright",
    ax=axes[2, 1]
)
axes[2, 1].set_title(
    "$\\mathbf{Andamento \\ temporale \\ del \\ numero \\ di \\ modelli \\ rilasciati}$\nper segmento di mercato (solo Amazon UK)",
    size=11
)
axes[2, 1].set(
    xlabel="Anno",
    ylabel="Numero di modelli rilasciati",
    facecolor="whitesmoke"
)
axes[2, 1].grid(
    ls="--",
    alpha=.5
)
axes[2, 1].legend(
    title="$\\mathbf{Segmento}$"
);

# Trasformazioni e rimozione degli outlier ⛔

## Applicazione del logaritmo naturale

### **Obiettivo**
Convertire le variabili numeriche con distribuzioni asimmetriche in distribuzioni più simmetriche, rendendo le relazioni tra variabili più lineari.

### **Formula**
$x_{\text{trasformato}} = \log(x + c)$
- $x$ è il valore originale.
- $c$ è una costante per evitare $\log(0)$ (solitamente $c=[0, 1]$).

### **Note** 
- Riduce la varianza.
- Evidenzia le relazioni lineari nascoste.
- Applicabile solo a valori positivi. Per gestire valori nulli o negativi, è necessario aggiungere una costante $c > 0$.

## Standardizzazione

### **Obiettivo**
Rendere le variabili numeriche comparabili, eliminando gli effetti delle diverse scale.

### **Formula:**
$x_{\text{standardizzato}} = \frac{x - \mu}{\sigma}$
- $x$ è il valore originale.
- $\mu$ è la media della variabile.
- $\sigma$ è la deviazione standard della variabile.

### **Note** 
  - Centra le variabili attorno a zero.
  - Rende la varianza unitaria.
  - Evita che variabili con scale maggiori dominino il modello.

In [ ]:
tr_data = data.copy()
scaler = {}

for var_i in num_vars + [target_var]:
    
    # applicazione del logaritmo
    x = np.log(np.array(tr_data[var_i]).reshape(-1, 1))

    # standardizzazione
    scaler[var_i] = StandardScaler()
    tr_data[var_i] = scaler[var_i].fit_transform(x)

## Rimozione degli outlier tramite IQR (Interquartile Range)

### **Obiettivo**
Eliminare valori estremi che potrebbero distorcere i risultati della regressione.

### **Procedimento**
  - $\text{IQR} = Q3 - Q1$.
  - $\text{lower-bound} = Q1 - 1.5 \times \text{IQR}$.
  - $\text{upper-bound} = Q3 + 1.5 \times \text{IQR}$.
  - Rimuovere i valori $x$ che soddisfano $x < Q1 - (1.5 \cdot \text{IQR}) \ \text{oppure} \ x > Q3 + (1.5 \cdot \text{IQR})$.

In [ ]:
for var_i in num_vars + [target_var]:
    
    if var_i != "cpu_cores":
        
        # calcolo dei quantili e dell'IQR
        Q1 = tr_data[var_i].quantile(0.25)
        Q3 = tr_data[var_i].quantile(0.75)
        IQR = Q3 - Q1
        
        # filtraggio
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        tr_data = tr_data[(tr_data[var_i] >= lower_bound) & (tr_data[var_i] <= upper_bound)]

## Confronto tra le distribuzioni dei dati originali e trasformati

In [ ]:
map = {
    "weight": "Peso [g]",
    "display_size": "Diagonale del display [inch]",
    "volume": "Volume [mm$^3$]",
    "chipset_production_process": "Processo produttivo [nm]",
    "cpu_cores": "Numero di core della CPU",
    "price_usd": "Prezzo di vendita [$]"
}

fig, axes = plt.subplots(
    ncols=4,
    nrows=len(num_vars + [target_var]),
    figsize=(18, 4.5*len(num_vars + [target_var]))
)
fig.subplots_adjust(
    wspace=.3,
    hspace=.45
)

for i, var_i in enumerate(num_vars + [target_var]):
    
    # istogramma dei dati originali
    sns.histplot(
        data=data,
        x=var_i,
        color="dodgerblue",
        stat="percent",
        bins=20,
        ax=axes[i, 0]
    )
    axes[i, 0].set_title(
        "$\\mathbf{Istogramma \\ dei \\ dati \\ originali}$\n" + map[var_i],
        size=11
    )
    axes[i, 0].set(
        xlabel=map[var_i],
        ylabel="Percentuale [%]",
        facecolor="whitesmoke"
    )
    axes[i, 0].grid(
        ls="--",
        alpha=.5
    )
    
    # distribuzione dei dati originali
    src_mu = round(data[var_i].mean(), 2)
    src_std = round(data[var_i].std(), 2)
    src_median = round(data[var_i].median(), 2)
    sns.boxenplot(
        data=data,
        x=var_i,
        color="dodgerblue",
        ax=axes[i, 1]
    )
    axes[i, 1].axvline(
        src_median,
        color="limegreen",
        ls="--",
        label="Mediana: " + str(src_median)
    )
    axes[i, 1].set_title(
        "$\\mathbf{Distribuzione \\ dei \\ dati \\ originali}$\n$\\mu=" + str(src_mu) + "$, $\\sigma=" + str(src_std) + "$",
        size=11
    )
    axes[i, 1].set(
        xlabel=map[var_i],
        facecolor="whitesmoke"
    )
    axes[i, 1].grid(
        axis="x",
        ls="--",
        alpha=.5
    )
    axes[i, 1].legend(
        loc="upper right"
    )
    
    # istogramma dei dati trasformati
    sns.histplot(
        data=tr_data,
        x=var_i,
        color="orangered",
        stat="percent",
        bins=20,
        kde=True,
        ax=axes[i, 2]
    )
    axes[i, 2].set_title(
        "$\\mathbf{Istogramma \\ dei \\ dati \\ trasformati}$\nOutlier filtrati",
        size=11
    )
    axes[i, 2].set(
        xlabel=map[var_i],
        ylabel="Percentuale [%]",
        facecolor="whitesmoke"
    )
    axes[i, 2].grid(
        ls="--",
        alpha=.5
    )
    
    # distribuzione dei dati trasformati
    tr_mu = round(tr_data[var_i].mean(), 2)
    tr_std = round(tr_data[var_i].std(), 2)
    tr_median = round(tr_data[var_i].median(), 2)
    sns.boxenplot(
        data=tr_data,
        x=var_i,
        color="orangered",
        ax=axes[i, 3]
    )
    axes[i, 3].axvline(
        tr_median,
        color="limegreen",
        ls="--",
        label="Mediana: " + str(tr_median)
    )
    axes[i, 3].set_title(
        "$\\mathbf{Distribuzione \\ dei \\ dati \\ trasformati}$\n$\\mu=" + str(tr_mu) + "$, $\\sigma=" + str(tr_std) + "$",
        size=11
    )
    axes[i, 3].set(
        xlabel=map[var_i],
        facecolor="whitesmoke"
    )
    axes[i, 3].grid(
        axis="x",
        ls="--",
        alpha=.5
    )

## Correlazioni prima e dopo

In [ ]:
fig, axes = plt.subplots(
    ncols=2,
    nrows=1,
    figsize=(15, 6)
)
fig.subplots_adjust(
    wspace=.5
)

# correlazioni tra le variabili originali
sns.heatmap(
    data[num_vars + [target_var]].corr(),
    cbar=False,
    annot=True,
    ax=axes[0]
)
axes[0].set_title(
    "$\\mathbf{Correlazioni \\ tra \\ le \\ variabili \\ originali}$",
    size=11,
    y=1.025
)
axes[0].set(
    yticklabels=map.values(),
    xticklabels=[]
)

# correlazioni tra le variabili trasformate
sns.heatmap(
    tr_data[num_vars + [target_var]].corr(),
    cbar=False,
    annot=True,
    ax=axes[1]
)
axes[1].set_title(
    "$\\mathbf{Correlazioni \\ tra \\ le \\ variabili \\ trasformate}$",
    size=11,
    y=1.025
)
axes[1].set(
    yticklabels=map.values(),
    xticklabels=[]
);

# Scelta dei regressori 😎

## Funzione per costruire la formula in stile R

In [ ]:
def build_formula(t_var, n_vars, c_vars):
    
    num_part = " + ".join(n_vars)
    cat_part = " + ".join([f"C({var_i})" for var_i in c_vars])
    formula_parts = [num_part, cat_part]
    formula = f"{t_var} ~ " + " + ".join([part_i for part_i in formula_parts if part_i])
    return formula

## Suddivisione del dataset in train e test

In [ ]:
tr_train_data, tr_test_data = train_test_split(tr_data, train_size=.8, random_state=1)

## Stepwise backward
L'approccio **stepwise backward** è una tecnica di selezione delle variabili utilizzata per costruire modelli statistici, come la regressione lineare, in modo da migliorare la parsimonia e ridurre l'overfitting. Questo metodo rimuove iterativamente le variabili meno significative (quelle che peggiorano maggiormente la qualità del modello) finché non si raggiunge una configurazione ottimale.

### **Procedimento**
1. **Si inizia con il modello completo**: si parte con un modello che include tutte le variabili (sia numeriche che categoriali) disponibili.
2. **Calcolo della metrica di selezione**: per ogni iterazione, viene calcolata una metrica che misura la bontà del modello, come l'`AIC (Akaike Information Criterion)` o il `BIC (Bayesian Information Criterion)`. Questi criteri bilanciano la qualità dell'adattamento e la complessità del modello.
3. **Rimozione della variabile meno significativa**: si esamina ogni variabile nel modello e si calcola l'impatto della sua rimozione. Si rimuove la variabile che comporta il miglior miglioramento del criterio selezionato (ad esempio, il valore AIC più basso).
4. **Iterazione**: il processo viene ripetuto fino a quando non è possibile rimuovere altre variabili senza peggiorare significativamente la qualità del modello. Il processo si ferma quando l'eliminazione di ulteriori variabili non porta a un miglioramento del modello.
5. **Modello finale**: il risultato finale è un modello con un numero ridotto di variabili che sono considerate le più rilevanti.

### **Note**
- Riduce il rischio di overfitting.
- Seleziona variabili significative per il modello, migliorando l'interpretabilità.

### **AIC vs BIC**
| **Criterio**         | **AIC**                              | **BIC**                              |
|:--------------------:|:------------------------------------:|:------------------------------------:|
| **Formula**          | $-2\ln(L) + 2k$                      | $-2\ln(L) + k\cdot\ln(n)$            |
| **Approccio**        | Basato sulla predizione futura       | Basato sull'inferenza bayesiana      |
| **Bias**             | Favorisce modelli complessi          | Favorisce modelli semplici           |
| **Dataset grande**   | Più permissivo                       | Penalizza severamente modelli complessi  |
| **Dataset piccolo**  | Più affidabile per evitare underfitting  | Più soggetto a underfitting          |

In [ ]:
def run_backward_stepwise(t_var, n_vars, c_vars, df, tolerance=1e-6):
    
    # inizializzazione della formula con tutte le variabili
    best_metric = smf.ols(
        formula=build_formula(t_var, n_vars, c_vars),
        data=df
    ).fit().aic
    current_vars = n_vars + c_vars
    print(f"AIC del modello completo: {round(best_metric, 2)}")

    while True:
        summary = pd.DataFrame()

        # calcolo dell'AIC per ogni possibile variabile da rimuovere
        for var in current_vars:
            vars_i = current_vars.copy()
            vars_i.remove(var)
            formula = build_formula(
                t_var,
                [v for v in vars_i if v in n_vars],
                [v for v in vars_i if v in c_vars]
            )
            metric_i = smf.ols(formula=formula, data=df).fit().aic
            summary = pd.concat(
                [
                    summary,
                    pd.DataFrame({"Removed variable": [var], "AIC": [metric_i]})
                ],
                ignore_index=True
            )
        
        # individuazione della variabile con l'AIC migliore
        best_candidate_idx = summary['AIC'].idxmin()
        best_candidate_aic = summary.loc[best_candidate_idx, "AIC"]
        var_to_remove = summary.loc[best_candidate_idx, "Removed variable"]
        
        # controllo: l'AIC è migliorato?
        if best_candidate_aic < best_metric - tolerance:
            best_metric = best_candidate_aic
            current_vars.remove(var_to_remove)
            print(f"- Variabile rimossa (AIC = {round(best_metric, 2)}): {var_to_remove}")
        else:
            break

    print(f"Variabili scelte:", sorted(current_vars))
    return [var for var in current_vars if var in n_vars], [var for var in current_vars if var in c_vars]

ch_num_vars, ch_cat_vars = run_backward_stepwise(target_var, num_vars.copy(), cat_vars.copy(), tr_train_data)

## Rimozione della multicollinearità tramite VIF

Il **VIF** (Variance Inflation Factor) consente di valutare la presenza di multicollinearità tra le variabili indipendenti in un modello di regressione.

### **Formula:**
$\text{VIF}_i = \frac{1}{1 - R_i^2}$
- $R_i^2$ rapresenta il coefficiente di determinazione della regressione della $i$-esima variabile indipendente sulle altre variabili indipendenti.

### **Interpretazione**
- $\text{VIF}_i = 1$: nessuna correlazione con le altre variabili.
- $1 < \text{VIF}_i \leq 5$: bassa correlazione, generalmente accettabile.
- $\text{VIF}_i > 5$: potenziale multicollinearità, da investigare.
- $\text{VIF}_i > 10$: multicollinearità severa, il modello potrebbe essere distorto.

In [ ]:
def VIF_var_selection(n_vars, df):
    
    # inizializzazione: tutti i regressori numerici
    print(f"Variabili numeriche iniziali: {sorted(n_vars)}")
    current_vars = n_vars
    
    while True:
        
        # calcolo del VIF per ogni covariata
        subset = df[current_vars]
        summary = pd.DataFrame()
        summary["Variable"] = subset.columns
        summary["VIF"] = [variance_inflation_factor(subset.values, i) for i in range(len(subset.columns))]
        
        # il regressore con il VIF peggiore viene rimosso
        if summary["VIF"].max() > 5:
            print(f"- Variabile rimossa (VIF = {round(summary["VIF"].max(),2)}): {summary.loc[summary["VIF"].idxmax(), "Variable"]}")
            current_vars.remove(summary.loc[summary["VIF"].idxmax(), "Variable"])
        else:
            break
    
    print(f"Variabili numeriche scelte:", sorted(current_vars))
    return current_vars

ch_num_vars = VIF_var_selection(ch_num_vars.copy(), tr_data)

# Stima di un modello di regressione multipla 👊

In [ ]:
mhat = smf.ols(
    formula=build_formula(target_var, ch_num_vars, ch_cat_vars),
    data=tr_train_data
).fit()
mhat.summary()

# Analisi dei residui 👀

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18, 4))
fig.subplots_adjust(
    wspace=.3,
    hspace=.45
)

residuals = mhat.resid

# test di normalità Jarque-Bera
jb, value = scipy.stats.jarque_bera(residuals)
sns.kdeplot(
    x=mhat.resid,
    color="dodgerblue",
    fill=True,
    alpha=.75,
    ax=axes[0]
)
sns.kdeplot(
    x=np.random.normal(loc=residuals.mean(), scale=residuals.std(),size=100000),
    color="gold",
    fill=True,
    alpha=.25,
    label="$N(" + str(round(residuals.mean(), 2)) + "; " + str(round(residuals.std(), 2)) + ")$",
    ax=axes[0]
)
axes[0].set_title(
    "$\\mathbf{Test \\ di \\ Jarque-Bera}$\n$JB=" + str(round(jb, 2)) + "$, $p-value=" + str(round(value*100, 2)) + "\\%$, accetto H0",
    size=11
)
axes[0].set(
    xlabel="Residuo trasf.",
    ylabel="Densità",
    facecolor="whitesmoke"
)
axes[0].grid(
    ls="--",
    alpha=.5,
)
axes[0].legend(
    loc="upper right"
)

# boxen-plot dei residui
sns.boxenplot(
    y=residuals,
    color="dodgerblue",
    ax=axes[1]
)
axes[1].set_title(
    "$\\mathbf{Boxen-plot \\ dei \\ residui}$\n$mediana=" + str(round(residuals.median(), 2)) + "$",
    size=11
)
axes[1].set(
    ylabel="Residuo trasf.",
    facecolor="whitesmoke"
)
axes[1].grid(
    ls="--",
    alpha=.5
)

# test di omoschedasticità Breusch-Pagan
_, pvalue, _, _ = het_breuschpagan(residuals, mhat.model.exog) 
sns.scatterplot(
    x=tr_train_data["price_usd"],
    y=residuals,
    color="dodgerblue",
    ax=axes[2]
)
axes[2].set_title(
    "$\\mathbf{Test \\ di \\ omoschedasticità}$\n$p-value=" + str(round(pvalue*100, 2)) + "\\%$, rifiuto H0",
    size=11
)
axes[2].set(
    ylabel="Residuo trasf.",
    xlabel="Prezzo di vendita trasf.",
    facecolor="whitesmoke"
)
axes[2].grid(
    ls="--",
    alpha=.5
)

# test di indipendenza Durbin-Watson
dw_statistic = durbin_watson(residuals)
plot_acf(
    residuals,
    lags=20,
    color="dodgerblue",
    ax=axes[3]
)
axes[3].set_title(
    "$\\mathbf{Test \\ di \\ indipendeza}$\n$statistica-dw=" + str(round(dw_statistic, 2)) + "$",
    size=11
)
axes[3].set(
    xlabel="Lag",
    ylabel="Autocorrelazione",
    ylim=(-.25, 1.25),
    facecolor="whitesmoke"
)
axes[3].grid(
    ls="--",
    alpha=.5
)

## Calcolo della leverage

In statistica e in particolare nell'analisi dei residui di un modello di regressione, la **leverage** è una misura di quanto i valori delle variabili indipendenti relative a un'osservazione sono lontani da quelli assunti dalle altre osservazioni. I punti a elevata leverage, se presenti, sono outlier rispetto alle variabili indipendenti.

### **Formula**
$\text{h}_{ii} = \text{H[i, i]}$
- $\text{H} = \text{X}(\text{X}^T\text{X})^{-1}\text{X}^T$ è la matrice di proiezione ortogonale.

### **Interpretazione**
- $\text{h}_{ii} > \frac{2(k+1)}{\text{N}}$: l'osservazione $i$-esima è un outlier capace di influenzare il piano di regressione.

In [ ]:
# calcolo della soglia e della leverage per ogni osservazione
th = 2*(mhat.df_model+1)/len(tr_test_data)
tr_train_data["leverage"] = mhat.get_influence().hat_matrix_diag
fig, axes = plt.subplots(
    ncols=2,
    nrows=1,
    figsize=(16, 5)
)

sns.scatterplot(
    data=tr_train_data,
    x="price_usd",
    y=residuals,
    hue=tr_train_data["leverage"],
    palette="crest",
    ax=axes[0]
)
sns.scatterplot(
    data=tr_train_data[tr_train_data.leverage >= th],
    x="price_usd",
    y=residuals,
    marker="x",
    s=100,
    color="black",
    label="Outlier",
    ax=axes[0]
)
axes[0].set_title(
    "$\\mathbf{Leverage \\ di \\ ogni \\ dato \\ di \\ train}$\n$th=" + str(round(th, 2)) + "$",
    size=11
)
axes[0].set(
    xlabel="Prezzo di vendita trasf.",
    ylabel="Residuo trasf.",
    facecolor="whitesmoke"
)
axes[0].grid(
    ls="--",
    alpha=.5
)

sns.kdeplot(
    x=tr_train_data["leverage"],
    fill=True,
    color="dodgerblue",
    ax=axes[1]
)
axes[1].axvline(
    th,
    color="orangered",
    ls="-.",
    label="Soglia"
)
axes[1].set_title(
    "$\\mathbf{Distribuzione \\ delle \\ leverage}$\n$mediana=" + str(round(tr_train_data.leverage.median(), 2)) + "$",
    size=11
)
axes[1].set(
    xlabel="Leverage",
    ylabel="Densità",
    facecolor="whitesmoke"
)
axes[1].grid(
    ls="--",
    alpha=.5
)
axes[1].legend();

# rimozione degli outlier
# tr_train_data = tr_train_data[tr_train_data.leverage < th]

[Torna al training del modello](#stepwise-backward)

# Validazione del modello 🍀

In [ ]:
# train
real_price_usd = np.exp(scaler["price_usd"].inverse_transform(np.array(tr_train_data["price_usd"]).reshape(-1, 1)))
predicted_price_usd = np.exp(scaler["price_usd"].inverse_transform(np.array(mhat.fittedvalues).reshape(-1, 1)))
train_RMSE = root_mean_squared_error(
    real_price_usd,
    predicted_price_usd
)
print(f"RMSE di train: {round(train_RMSE, 2)} $")

# test
real_price_usd = np.exp(scaler["price_usd"].inverse_transform(np.array(tr_test_data["price_usd"]).reshape(-1, 1)))
predicted_price_usd = np.exp(scaler["price_usd"].inverse_transform(np.array(mhat.predict(tr_test_data)).reshape(-1, 1)))
test_RMSE = root_mean_squared_error(
    real_price_usd,
    predicted_price_usd
)
print(f"RMSE di test: {round(test_RMSE, 2)} $")

# Approfondimento: le reti neurali (NN) con `tensorflow`🔥

Una **rete neurale** è un modello computazionale ispirato al funzionamento del cervello umano, progettato per elaborare dati e apprendere schemi complessi. È composta da **neuroni artificiali**, organizzati in **strati**:

1. **Strato di input**: riceve i dati iniziali.
2. **Strati nascosti**: elaborano le informazioni attraverso operazioni matematiche e funzioni di attivazione.
3. **Strato di output**: restituisce il risultato finale.

Ogni connessione tra i neuroni ha un **peso**, che viene ottimizzato durante l'addestramento per migliorare le prestazioni del modello. Questo tipo di rete è usato in compiti come classificazione, regressione e predizione.

### **Modello**
<img src="https://www.researchgate.net/profile/Ali-Polat-9/publication/322276434/figure/fig2/AS:623915986587648@1525764571315/Structure-of-artificial-neuron.png">

### **Esempio**
<img src="https://miro.medium.com/v2/resize:fit:800/0*-7I6evZetK0o_hq0.png">

# Importazione dei pacchetti

In [ ]:
from keras.models import Sequential
from tensorflow.keras.models import load_model
from keras.layers import Dense, Normalization, Input, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

## Codifica One-Hot delle variabili categoriche

In [ ]:
encoded_data = pd.get_dummies(
    data=data,
    columns=cat_vars,
    drop_first=True
).astype(float)
encoded_data.head(5)

## Suddivisione del dataset in train e test

In [ ]:
X = encoded_data.drop(columns="price_usd")
y = encoded_data["price_usd"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=.3,
    random_state=1
)

## Definizione del modello

In [ ]:
try:
    model = load_model("NN.keras")
except:
    model = Sequential(
        [
            Input(shape=(X_train.shape[1], )),
            Normalization(),
            Dense(16),
            Dropout(.25),
            Dense(8),
            Dense(1)
        ]
    )
    model.compile(
        optimizer="adam",
        loss="mean_squared_error"
    )
    model.summary()

## Stima del modello

In [ ]:
# funzioni di callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=500,
    restore_best_weights=True
)
checkpoint = ModelCheckpoint(
    filepath="NN.keras",
    monitor="val_loss",
    save_best_only=True,
    mode="min",
    save_weights_only=False,
    verbose=1
)

model.fit(
    X_train,
    y_train,
    epochs=10000,
    batch_size=64,
    validation_split=.3,
    callbacks=[
        checkpoint,
        early_stop
    ]
)

## Validazione del modello

In [ ]:
# train
best_model = load_model("NN.keras")
y_pred_train = best_model.predict(X_train).reshape(-1)
test_RMSE = root_mean_squared_error(
    y_true=np.array(y_train),
    y_pred=y_pred_train
)
print(f"RMSE di train: {round(test_RMSE, 2)} $")

# test
best_model = load_model("NN.keras")
y_pred_test = best_model.predict(X_test).reshape(-1)
test_RMSE = root_mean_squared_error(
    y_true=np.array(y_test),
    y_pred=y_pred_test
)
print(f"RMSE di test: {round(test_RMSE, 2)} $")

### Distribuzione dei residui

In [ ]:
fig, axes = plt.subplots(
    ncols=2,
    nrows=1,
    figsize=(16, 5)
)

# train
sns.histplot(
    x = np.array(y_train) - y_pred_train,
    color="dodgerblue",
    stat="percent",
    kde=True,
    ax=axes[0]
)
axes[0].set_title(
    "$\\mathbf{Distribuzione \\ dei \\ residui \\ di \\ train}$",
    size=11
)
axes[0].set(
    xlabel="Residuo [°C]",
    ylabel="Percentuale [%]",
    xlim=(-1000, 1000),
    facecolor="whitesmoke"
)
axes[0].grid(
    ls="--",
    alpha=.5
)

# test
sns.histplot(
    x = np.array(y_test) - y_pred_test,
    color="orangered",
    stat="percent",
    kde=True,
    ax=axes[1]
)
axes[1].set_title(
    "$\\mathbf{Distribuzione \\ dei \\ residui \\ di \\ test}$",
    size=11
)
axes[1].set(
    xlabel="Residuo [°C]",
    ylabel="Percentuale [%]",
    xlim=(-1000, 1000),
    facecolor="whitesmoke"
)
axes[1].grid(
    ls="--",
    alpha=.5
)